In [ ]:
pip install psycopg2-binary

In [ ]:
pip install unidecode

In [ ]:
import csv
import os
import zipfile

import psycopg2
import psycopg2.extras
import unidecode
import requests
import re

In [ ]:
conn = psycopg2.connect(database="campaign-finance-new",
                        user="data",
                       password="data",
                        host="postgresdb",
                        port="5432")
c = conn.cursor()

## **Raw Tables**

These are the files downloaded from the SBOE site transaction search, they only have to be imported once


In [ ]:
print('creating raw_table table...')
c.execute("CREATE TABLE raw_table "
          "(trans_id SERIAL PRIMARY KEY, "
          " name VARCHAR(200) DEFAULT '', "
          " street_line_1 VARCHAR(200) DEFAULT '', street_line_2 VARCHAR(200) DEFAULT '', "
          " city VARCHAR(200) DEFAULT '', state VARCHAR(15) DEFAULT '', "
          " zip_code VARCHAR(11) DEFAULT '', occupation VARCHAR(200) DEFAULT '', "
          " employer VARCHAR(200) DEFAULT '', transaction_type VARCHAR(200) DEFAULT '', "
          " transaction_category VARCHAR(10) DEFAULT '', committee_name VARCHAR(200) DEFAULT '', "
          " committee_sboe_id VARCHAR(200) DEFAULT '', committee_street_1 VARCHAR(200) DEFAULT '', "
          " committee_street_2 VARCHAR(200) DEFAULT '', committee_city VARCHAR(200) DEFAULT '', "
          " committee_state VARCHAR(200) DEFAULT '', committee_zip_code VARCHAR(200) DEFAULT '', "
          " report_name character varying(200) DEFAULT '', date_occured VARCHAR(200) DEFAULT '', "
          " account_code VARCHAR(200) DEFAULT '', amount VARCHAR(200) DEFAULT '', "
          " form_of_payment VARCHAR(200) DEFAULT '', purpose VARCHAR(500) DEFAULT '', "
          " candidate_referendum_name VARCHAR(200) DEFAULT '', declaration VARCHAR(200) DEFAULT '', "
          " import_date DATE DEFAULT CURRENT_DATE"
          " )")

print("Done.")
conn.commit()



In [ ]:
directory = os.fsencode("./data/raw_files/contributions")
    
for filename in os.listdir(directory):
    full_filename = os.path.join(directory, filename)
    print(full_filename)
    with open(full_filename, 'rU') as csv_file: 
        c.copy_expert("COPY raw_table " 
                      "(name, street_line_1, street_line_2, city, state, " 
                      " zip_code, occupation, employer, transaction_type, " 
                      " committee_name, committee_sboe_id, committee_street_1, " 
                      " committee_street_2, committee_city, committee_state, " 
                      "committee_zip_code, report_name, date_occured, account_code, " 
                      " amount, form_of_payment, purpose, candidate_referendum_name," 
                      " declaration) " 
                      "FROM STDIN CSV HEADER", csv_file)

    conn.commit()

print("Done.")

In [ ]:
print('fixing transaction categories ...')
c.execute("update raw_table "
          "set transaction_category = 'C' "
          )
print("Done.")

In [ ]:
conn.commit()

In [ ]:
directory = os.fsencode("./data/raw_files/expenses")
    
for filename in os.listdir(directory):
    full_filename = os.path.join(directory, filename)
    print(full_filename)
    with open(full_filename, 'rU') as csv_file: 
        c.copy_expert("COPY raw_table " 
                      "(name, street_line_1, street_line_2, city, state, " 
                      " zip_code, occupation, employer, transaction_type, " 
                      " committee_name, committee_sboe_id, committee_street_1, " 
                      " committee_street_2, committee_city, committee_state, " 
                      "committee_zip_code, report_name, date_occured, account_code, " 
                      " amount, form_of_payment, purpose, candidate_referendum_name," 
                      " declaration) " 
                      "FROM STDIN CSV HEADER", csv_file)

    conn.commit()

print("Done.")

In [ ]:
print('fixing transaction categories ...')
c.execute("update raw_table "
          "set transaction_category = 'E' "
          " WHERE transaction_category <> 'C' "
          )
print("Done.")

In [ ]:
conn.commit()

## **Transaction Tables**

We are parsing out the information from the raw tables into specific objects 

In [ ]:
print('creating accounts table...')
c.execute("CREATE TABLE accounts "
          "(account_id SERIAL PRIMARY KEY, "
          " name VARCHAR(200) DEFAULT '' NOT NULL, "
          " address_1 VARCHAR(200) DEFAULT '' NOT NULL, address_2 VARCHAR(200) DEFAULT '' NOT NULL, "
          " city VARCHAR(200) DEFAULT '' NOT NULL, state VARCHAR(15) DEFAULT '' NOT NULL, "
          " zip VARCHAR(11) DEFAULT '' NOT NULL, occupation VARCHAR(200) DEFAULT '' NOT NULL, "
          " employer VARCHAR(200) DEFAULT '' NOT NULL, "
          " is_donor INT, is_vendor INT, is_person INT, is_organization INT,"
          " lat REAL, lon REAL, census_block BIGINT, import_date TIMESTAMP"
          " )")
print("table created, inserting records")
c.execute("INSERT INTO accounts "
          "(name, address_1, "
          " address_2, city, state, zip, occupation, employer, import_date) "
          "SELECT DISTINCT "
          "COALESCE(UPPER(TRIM(name)), ''), "
          "COALESCE(UPPER(TRIM(street_line_1)), ''), COALESCE(UPPER(TRIM(street_line_2)), ''), "
          "COALESCE(UPPER(TRIM(city)), ''), COALESCE(UPPER(TRIM(state)), ''), COALESCE(UPPER(TRIM(zip_code)), ''), "
          "COALESCE(UPPER(TRIM(occupation)), ''), COALESCE(UPPER(TRIM(employer)), ''), import_date "
          "FROM raw_table")

print("Done.")
conn.commit()

In [ ]:
print('creating indexes on accounts table...')
c.execute("CREATE INDEX accounts_account_info ON accounts "
          "(name, address_1, address_2, city, "
          " state, zip)")
print("Done.")
conn.commit()

In [ ]:
conn.commit()

In [ ]:
print('creating committees table...')
c.execute("CREATE TABLE committees "
          "(comm_id SERIAL PRIMARY KEY, "
          " sboe_committee_id VARCHAR(200), name VARCHAR(200), "
          " address_1 VARCHAR(200), address_2 VARCHAR(200), "
          " city VARCHAR(200), state VARCHAR(200), zip VARCHAR(200), "
          " candidate_id INT, treasurer_id INT, asst_treasurer_id INT, import_date TIMESTAMP "
          ")")
print("inserting records")
c.execute("INSERT INTO committees "
          "(sboe_committee_id, name, address_1, address_2, city, state, zip, import_date) "
          "SELECT DISTINCT COALESCE(UPPER(TRIM(committee_sboe_id)), ''), "
          "COALESCE(UPPER(TRIM(committee_name)), ''), COALESCE(UPPER(TRIM(committee_street_1)), ''), COALESCE(UPPER(TRIM(committee_street_2)), ''), "
          "COALESCE(UPPER(TRIM(committee_city)), ''), COALESCE(UPPER(TRIM(committee_state)), ''), COALESCE(UPPER(TRIM(committee_zip_code)), ''), import_date "
          "FROM raw_table")

print("Done.")
conn.commit()

In [ ]:
print('creating transactions table...')
c.execute("CREATE TABLE transactions "
                "(transaction_id INT, original_committee_sboe_id VARCHAR(200), original_account_id INT,  "
                " transaction_type VARCHAR(200), transaction_category VARCHAR(10), date_occured DATE, amount FLOAT, "
                " report_name VARCHAR(200), account_code VARCHAR(200), form_of_payment VARCHAR(200), "
                " purpose VARCHAR(500), candidate_referendum_name VARCHAR(200), declaration VARCHAR(200), "
                "canon_account_id INT, canon_committee_sboe_id VARCHAR(200), import_date TIMESTAMP "
                ")")
print("Done.")
conn.commit()

In [ ]:
print("Inserting transaction records")
c.execute("INSERT INTO transactions "
          "(transaction_id, original_committee_sboe_id, original_account_id, "
          " transaction_type, transaction_category, date_occured, amount, report_name, account_code, "
          " form_of_payment, purpose, candidate_referendum_name, declaration)"
          "SELECT trans_id, "
          "COALESCE(UPPER(TRIM(committee_sboe_id)), ''), "
          "accounts.account_id, "
          "COALESCE(UPPER(TRIM(transaction_type)), ''), "
          "COALESCE(UPPER(TRIM(transaction_category)), ''), "
          "TO_DATE(TRIM(date_occured), 'MM/DD/YYYY'), "
          "CAST(amount as DOUBLE PRECISION), "
          "COALESCE(UPPER(TRIM(report_name)), ''), "
          "COALESCE(UPPER(TRIM(account_code)), ''), "
          "COALESCE(UPPER(TRIM(form_of_payment)), ''), "
          "COALESCE(UPPER(TRIM(purpose)), ''), "
          "COALESCE(UPPER(TRIM(candidate_referendum_name)), ''), "
          "COALESCE(UPPER(TRIM(declaration)), '') "
          "FROM raw_table LEFT JOIN accounts ON "
          "accounts.name = COALESCE(UPPER(TRIM(raw_table.name)), '') AND "
          "accounts.address_1 = COALESCE(UPPER(TRIM(raw_table.street_line_1)), '') AND "
          "accounts.address_2 = COALESCE(UPPER(TRIM(raw_table.street_line_2)), '') AND "
          "accounts.city = COALESCE(UPPER(TRIM(raw_table.city)), '') AND "
          "accounts.state = COALESCE(UPPER(TRIM(raw_table.state)), '') AND "
          "accounts.employer = COALESCE(UPPER(TRIM(raw_table.employer)), '') AND "
          "accounts.occupation = COALESCE(UPPER(TRIM(raw_table.occupation)), '') AND "
          "accounts.zip = COALESCE(UPPER(TRIM(raw_table.zip_code)), '')")
print("Done.")
conn.commit()

In [ ]:
print('creating indexes on transactions...')
c.execute("ALTER TABLE transactions ADD PRIMARY KEY(transaction_id)")
c.execute("CREATE INDEX donor_idx ON transactions (original_account_id)")
c.execute("CREATE INDEX committee_idx ON transactions (original_committee_sboe_id)")
c.execute("CREATE INDEX canon_donor_idx ON transactions (canon_account_id)")
print("Done.")
conn.commit()

In [ ]:
print('creating processed_accounts...')
c.execute("CREATE TABLE processed_accounts AS "
          "(SELECT account_id, "
          " CASE WHEN (name = '') "
          "      THEN NULL "
          "      ELSE LOWER(CONCAT_WS(' ', name)) "
          " END AS name, "  
           " CASE WHEN (address_1 = '' AND address_2 = '') "
          "      THEN NULL "
          "      ELSE LOWER(CONCAT_WS(' ', address_1, address_2)) "
          " END AS address, " 
          " CASE WHEN (city = '') "
          "      THEN NULL "
          "      ELSE LOWER(city) "
          " END AS city, "
          " CASE WHEN (state = '') "
          "      THEN NULL "
          "      ELSE LOWER(state) "
          " END AS state, "
          " CASE WHEN (zip = '') "
          "      THEN NULL "
          "      ELSE LOWER(zip) "
          " END AS zip, "
          " CASE WHEN (occupation = '') "
          "      THEN NULL "
          "      ELSE LOWER(occupation) " 
          " END AS occupation, "
          " CASE WHEN (employer = '') "
          "      THEN NULL "
          "      ELSE LOWER(employer) "
          " END AS employer "
        " FROM accounts)")

c.execute("CREATE INDEX processed_account_idx ON processed_accounts (account_id)")
print("Done.")
conn.commit()

In [ ]:
print('creating processed_committees...')
c.execute("CREATE TABLE processed_committees AS "
          "(SELECT comm_id, "
          " CASE WHEN (name = '') "
          "      THEN NULL "
          "      ELSE LOWER(CONCAT_WS(' ', name)) "
          " END AS name, "  
           " CASE WHEN (address_1 = '' AND address_2 = '') "
          "      THEN NULL "
          "      ELSE LOWER(CONCAT_WS(' ', address_1, address_2)) "
          " END AS address, " 
          " LOWER(city) AS city, "
          " LOWER(state) AS state, "
          " LOWER(zip) AS zip "
          " FROM committees)")

c.execute("CREATE INDEX processed_committee_idx ON processed_committees (comm_id)")
print("Done.")
conn.commit()